# Halloween challenge
---

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

# Code
---

### Imported Libraries

In [1]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from copy import copy

### Problem instance

This function, called ``make_set_covering_problem``, returns a sparse matrix of size ``num_sets`` (number of sets) * ``num_points`` (number of elements) with Boolean values (True or False) in which the rows represent sets, and the columns represent the elements covered by the sets. The density of the matrix is controlled by a parameter called ``density``. For each pair, if a random number generated with random() is less than the specified ``density``, sets the corresponding element to True in the set.

The ``current_state`` is an array of boolean where true indicates if the state contains that particular set, false if it doesn't contain it. We have to initialize it to a random possible solution.

In [2]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

In [11]:
num_points = 1000
num_sets = num_points
density = .3
matrix = make_set_covering_problem(num_points, num_sets, density)

starting_state = [choice([True, False]) for _ in range(num_sets)]

### Fitness function

The ``fitness`` function return the number of covered elements and the negative cost that is the number of taken sets as negative. That's because we want to take the solution with the smallest number of taken sets.

In [4]:
def fitness(state):
    global fitness_counter
    fitness_counter+=1
    cost = sum(state)
    valid = len(list(set(np.concatenate(matrix.rows[state]))))
    return valid, -cost

# Hill Climbing

The ``tweak`` function swap one of the set randomly, if it was taken we change it into not taken and vice versa.

Results:

| num points | num sets | density | number of evaluation | best fitness |
|------------|----------|---------|----------------------|--------------|
| 100        |    100   | .3      | 200                  | (100, -14)   |
| 100        | 100      | .7      | 200                  | (100, -21)   |
| 1000       | 1000     | .3      | 2.000                | (1000, -183) |
| 1000       | 1000     | .7      | 2.000                | (1000, -192) |
| 5000       | 5000     | .3      | 10.000               | (5000, -929) |
| 5000       | 5000     | .7      | 10.000               | (5000, -)    |

In [5]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, num_points - 1) # pick a random index
    new_state[index] = not new_state[index] # swap
    return new_state

In [ ]:
current_state = starting_state
global fitness_counter
fitness_counter=0
for step in range(num_points):  
    new_state = tweak(current_state)
    if fitness(new_state) > fitness(current_state):
        current_state = new_state
print(fitness(current_state), fitness_counter)

# Simulated Annealing
This algorithm is a Hill Climbing but with probability $p≠0$ of accepting a worsening solution $s'$ where $f(s)>f(s)': p=e^- \frac{f(s)-f(s')}{t}$ where $s$ is the current solution,  $s'$ is the tweaked one and $t$ is the temperature. The idea is that the further we go with the exploration, the more T decreases and with it also the probability of accepting worse solutions.

Results:

| num points | num sets | density | number of evaluation | best fitness  |
|------------|----------|---------|----------------------|---------------|
| 100        |    100   | .3      | 200                  | (100, -38)    |
| 100        | 100      | .7      | 200                  | (100, -45)    |
| 1000       | 1000     | .3      | 2.000                | (1000, -488)  |
| 1000       | 1000     | .7      | 2.000                | (1000, -492)  |
| 5000       | 5000     | .3      | 10.000               | (5000, -2461) |
| 5000       | 5000     | .7      | 10.000               | (5000, -2511) |

In [ ]:
current_state = starting_state
global fitness_counter
fitness_counter=0
t = num_points
for step in range(num_points):
    new_state = tweak(current_state)
    _, newCost = fitness(new_state)
    _, oldCost = fitness(current_state)
    if newCost <= oldCost and random() < np.exp(-((oldCost-newCost)/t)):
        current_state = new_state
    elif newCost > oldCost:
        current_state = new_state
    t=t-1    
print((fitness(current_state), fitness_counter))

# Tabu Search
This algorithm keeps track of the states we have visited and avoids going back to them.

Results:

| num points | num sets | density | number of evaluation | best fitness |
|------------|----------|---------|----------------------|--------------|
| 100        |    100   | .3      | 191                  | (100, -14)   |
| 100        | 100      | .7      | 197                  | (100, -21)   |
| 1000       | 1000     | .3      | 1.993                | (1000, -183) |
| 1000       | 1000     | .7      | 1.991                | (1000, -190) |
| 5000       | 5000     | .3      | 9.997                | (5000, -929) |
| 5000       | 5000     | .7      | 9.991                | (5000, -941) |

In [12]:
current_state = starting_state
already_visited = []
global fitness_counter
fitness_counter=0
for step in range(num_points):
    new_state = tweak(current_state)
    if new_state not in already_visited:
        already_visited.append(new_state)
        if fitness(new_state) > fitness(current_state):
            current_state = new_state
print(fitness(current_state), fitness_counter)

(1000, -183) 1993


# Iterated Local Search
Is a version of Hill Climbing. The Hill Climbing algorithm is put into a loop and restarted in a new position which can be the global optimum or the last optimum.

Results: 

| num points | num sets | density | number of evaluation | best fitness |
|------------|----------|---------|----------------------|--------------|
| 100        |    100   | .3      | 200                  | (100, -9)    |
| 100        | 100      | .7      | 200                  | (100, -4)    |
| 1000       | 1000     | .3      | 2.000                | (1000, -14)  |
| 1000       | 1000     | .7      | 5.000                | (1000, -5)   |
| 5000       | 5000     | .3      | 100.000              | (5000, -21)  |
| 5000       | 5000     | .7      | 100.000              | (5000, -11)  |

In [ ]:
current_state = starting_state
global fitness_counter
fitness_counter=0
N_Restart = 5
solution = current_state
for i in range (N_Restart):
    for step in range(num_points):
        new_state = tweak(solution)
        if fitness(new_state) > fitness(solution):
            solution = new_state
print((fitness(solution), fitness_counter))